In [13]:
import requests
from bs4 import BeautifulSoup

### 除去id以後的參數，得到乾淨的網址

In [ ]:
url_raw=input('請輸入網址：')
url=url_raw.split('&')[0]
url

In [10]:
url_raw='http://hylib.ht.org.tw/bookDetail.do?id=104838'
url=url_raw.split('&')[0]
url

'http://hylib.ht.org.tw/bookDetail.do?id=104838'

### 從網址中再擷取書id，以後續填入data

In [11]:
try:
    book_id=url.split('=')[1]
except :
    print('請填入符合圖書館格式之網址')
else:print('書的id為',book_id,type(book_id))

書的id為 104838 <class 'str'>


### 取得書名資料
需要抓整頁，只在新增時進行！

In [4]:
book_page = requests.get(url)
book_page_soup = BeautifulSoup(book_page.text, 'html.parser')

book_name=book_page_soup.find('h3').text
book_name

'孤獨六講'

### 取得書的借閱狀態詳細資料
只要表格內容就好，不用欄位名稱

In [14]:
data = {'id':book_id}

resp = requests.post(
    'http://hylib.ht.org.tw/maintain/HoldListForBookDetailAjax.do', data=data)
soup = BeautifulSoup(resp.text, 'html.parser')
soup


<div id="holdListLoading"></div>
		
		
		排序 
		<select id="holdSortType" onchange="changeHoldListSortType()">
<option selected="selected" value="">請選擇</option>
<option value="h.status-asc">館藏狀態-升冪</option>
<option value="h.status-desc">館藏狀態-降冪</option>
<option value="h.barcode-asc">條碼號-升冪</option>
<option value="h.barcode-desc">條碼號-降冪</option>
<option value="c.collection_code,f.feature_code,class_no,author_no,description3,description,description2-asc,description4-asc">索書號-升冪</option>
<option value="c.collection_code desc,f.feature_code desc,class_no desc,author_no desc,description3 desc,description desc,description2-desc,description4-desc">索書號-降冪</option>
</select>
		總件數 <span class="red" id="holdListAllNum">2</span>件  
		<img alt="上10頁" border="0" onclick="holdListMovePage('-',10)" src="images/backall.gif" style="cursor:pointer"/>  
		<img alt="上一頁" border="0" onclick="holdListMovePage('-',1)" src="images/back01.gif" style="cursor:pointer"/>   			
		
				<a style="color:red">1</a>
<i

In [6]:
table=soup.find('table', 'order')
table

<table class="order">
<tr>
<th>#</th>
<th>條碼號</th>
<th>館藏地/室</th>
<th>索書號</th>
<th>資料類型</th>
<th>目前狀態 / 到期日</th>
<th>附件</th>
<th rel="reserve">預約</th>
</tr>
<tr>
<td>1</td>
<td>0312556</td>
<td>敦化總館/敦化總館</td>
<td> 855 4424-1  [+] </td>
<td>附件/一般</td>
<td>書在館</td>
<td></td>
<td rel="reserve">
				
								不可預約
					
							</td>
</tr>
<tr>
<td>2</td>
<td>0312555</td>
<td>敦化總館/敦化總館</td>
<td> 855 4424-1   </td>
<td>一般圖書/一般</td>
<td>書在館</td>
<td>附光碟1片</td>
<td rel="reserve">
				
								不可預約
					
							</td>
</tr>
</table>

In [20]:
books = []
for tr in table.find_all('tr')[1:]:
    book = []
    book_dict = {}
    book_key = [
        'book_name', 'book_id', 'copy', 'barcode_id', 'location',
        'call_number', 'data_type', 'status', 'reservation'
    ]
    book.append(book_name)
    book.append(book_id)
    for td in tr.find_all('td'):
        if td.text == '':
            continue
        book.append(td.text.strip().strip('/').strip())
        if len(book) > 9:
            book.pop(-2)
        book_dict = dict(zip(book_key, book))
    books.append(book_dict)

In [18]:
books = []
for tr in table.find_all('tr')[1:]:
    book = []
    book.append(book_name)
    book.append(book_id)
    for td in tr.find_all('td'):
        if td.text == '':
            continue
        book.append(td.text.strip().strip('/').strip())
        if len(book) > 9:
            book.pop(-2)
    books.append(book)

In [29]:
for td in table.find_all('tr')[1].find_all('td'):
    print(td.text.strip().strip('/').strip(),'○')

1 ○
0421610 ○
敦化總館/敦化總館 ○
312.932P97 4681 ○
一般圖書/一般 ○
已被外借 / 2019-04-01 ○
 ○
0人預約 ○


In [21]:
books

[{'book_name': '孤獨六講',
  'book_id': '104838',
  'copy': '1',
  'barcode_id': '0312556',
  'location': '敦化總館/敦化總館',
  'call_number': '855 4424-1  [+]',
  'data_type': '附件/一般',
  'status': '書在館',
  'reservation': '不可預約'},
 {'book_name': '孤獨六講',
  'book_id': '104838',
  'copy': '2',
  'barcode_id': '0312555',
  'location': '敦化總館/敦化總館',
  'call_number': '855 4424-1',
  'data_type': '一般圖書/一般',
  'status': '書在館',
  'reservation': '不可預約'}]

In [1]:
import re

In [8]:
re.match(r'\d+','333一').group()

'333'